<div dir=ltr align=center>
<font color=0F5298 size=10>
    Massive Data Mining <br>
<font color=0F5298 size=5>
    Electrical Engineering Department <br>
    Fall 2024 <br>
    Parham Gilani - 400101859 <br>
    Homework 2 - Spark Exercise <br>
    
____

# Import Libraries

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from hashlib import sha256
from pyspark.ml.feature import StopWordsRemover
from itertools import combinations
import hashlib
import random
import json
import re
from pyspark.sql import SparkSession
import numpy as np
import warnings 

warnings.filterwarnings('ignore') 
spark = SparkSession \
    .builder \
    .appName("MDA2024-HW2") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

# Part 1:

## Preprocessing:

In [3]:
arxiv_rdd = sc.textFile("MDA2024-Arxiv-Dataset.json")
arxiv_rdd.take(10)

['{"id":"0704.0001","submitter":"Pavel Nadolsky","authors":"C. Bal\\\\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","title":"Calculation of prompt diphoton production cross sections at Tevatron and\\n  LHC energies","comments":"37 pages, 15 figures; published version","journal-ref":"Phys.Rev.D76:013009,2007","doi":"10.1103/PhysRevD.76.013009","report-no":"ANL-HEP-PR-07-12","categories":"hep-ph","license":null,"abstract":"  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatro

In [4]:
# Parse the JSON string
def parse_json(line):
    return json.loads(line)

parsed_rdd = arxiv_rdd.map(parse_json).filter(lambda x: x is not None)

In [5]:
# Identify and remove or impute null values
def remove_nulls(record):
    # Check for null or empty fields in the record
    if all(record.values()):  # Ensure no field is None or empty
        return record
    else:
        return None  # Ignore records with null or empty fields

cleaned_rdd = parsed_rdd.filter(lambda x: remove_nulls(x) is not None)

In [6]:
# Define a set of stopwords for removal
stopwords = set(StopWordsRemover.loadDefaultStopWords("english"))

def remove_stopwords(record):
    for key, value in record.items():
        if isinstance(value, str):  # Process only text fields
            words = value.split()
            filtered_words = [word for word in words if word.lower() not in stopwords]
            record[key] = " ".join(filtered_words)
    return record

stopwords_removed_rdd = cleaned_rdd.map(remove_stopwords)

In [7]:
def remove_useless_characters(record):
    for key, value in record.items():
        if isinstance(value, str):  # Process only text fields
            # Remove special characters, retaining only alphanumeric and spaces
            record[key] = re.sub(r"[^a-zA-Z0-9\s]", "", value)
    return record

final_cleaned_rdd = stopwords_removed_rdd.map(remove_useless_characters)

# Save or view the final cleaned RDD
final_cleaned_rdd.take(1)  # View the first cleaned records

[{'id': '07040008',
  'submitter': 'Damian Swift',
  'authors': 'Damian C Swift',
  'title': 'Numerical solution shock ramp compression general material properties',
  'comments': 'Minor corrections',
  'journal-ref': 'Journal Applied Physics vol 104 073536 2008',
  'doi': '10106312975338',
  'report-no': 'LAUR072051 LLNLJRNL410358',
  'categories': 'condmatmtrlsci',
  'license': 'httparxivorglicensesnonexclusivedistrib10',
  'abstract': 'general formulation developed represent material models applications dynamic loading Numerical methods devised calculate response shock ramp compression ramp decompression generalizing previous solutions scalar equations state numerical methods found flexible robust matched analytic results high accuracy basic ramp shock solution methods coupled solve composite deformation paths shockinduced impacts shock interactions planar interface different materials calculations capture much physics typical material dynamics experiments without requiring spatiall

In [8]:
# Get a 1% sample from final_cleaned_rdd
sample_fraction = 0.15
sampled_rdd = final_cleaned_rdd.sample(withReplacement=False, fraction=sample_fraction, seed=42)

# Check how many records are in the sampled RDD
sampled_count = sampled_rdd.count()
print(f"Number of records in sampled RDD ({sample_fraction*100}%): {sampled_count}")

Number of records in sampled RDD (15.0%): 6091


## A:

In [9]:
# Parameters
first_pass_support_threshold = 300  # Support threshold for frequent words in the first pass
second_pass_support_threshold = 1000  # Support threshold for frequent trigrams in the second pass

# First pass: Count frequent words
def first_pass_apriori(rdd, support_threshold):
    # Count occurrences of words
    word_counts = rdd.flatMap(lambda record: record['abstract'].split()) \
                     .map(lambda word: (word, 1)) \
                     .reduceByKey(lambda x, y: x + y)
    
    # Find frequent words
    frequent_words = word_counts.filter(lambda x: x[1] >= support_threshold) \
                                .map(lambda x: x[0]) \
                                .collect()
    frequent_word_set = set(frequent_words)
    return frequent_word_set

# Second pass: Generate and count candidate trigrams
def second_pass_apriori(rdd, frequent_word_set, support_threshold):
    # Generate and count trigrams
    candidate_trigrams = rdd.flatMap(
        lambda record: [
            tuple(sorted(trigram))
            for trigram in combinations(record['abstract'].split(), 3)
            if len(set(trigram)) == 3 and set(trigram).issubset(frequent_word_set)
        ]
    ).map(lambda trigram: (trigram, 1)) \
     .reduceByKey(lambda x, y: x + y)

    # Filter trigrams based on support threshold
    frequent_trigrams = candidate_trigrams.filter(lambda x: x[1] >= support_threshold)
    return frequent_trigrams

# First pass: Find frequent words
frequent_word_set = first_pass_apriori(sampled_rdd, first_pass_support_threshold)

# Second pass: Generate and filter frequent trigrams
frequent_trigrams_apriori = second_pass_apriori(sampled_rdd, frequent_word_set, second_pass_support_threshold)

# Collect and display results
results = frequent_trigrams_apriori.collect()
for trigram, count in results:
    print(f"Trigram: {trigram}, Count: {count}")


Trigram: ('dark', 'interactions', 'matter'), Count: 1682
Trigram: ('TeV', 'mass', 'search'), Count: 1084
Trigram: ('dark', 'matter', 'parameter'), Count: 1194
Trigram: ('GeV', 'boson', 'mass'), Count: 1220
Trigram: ('dark', 'matter', 'symmetry'), Count: 1381
Trigram: ('dark', 'matter', 'searches'), Count: 1510
Trigram: ('Higgs', 'boson', 'vector'), Count: 1132
Trigram: ('Higgs', 'TeV', 'boson'), Count: 1781
Trigram: ('LHC', 'dark', 'matter'), Count: 1109
Trigram: ('TeV', 'mass', 'model'), Count: 1049
Trigram: ('Higgs', 'boson', 'bosons'), Count: 1251
Trigram: ('dark', 'matter', 'two'), Count: 1616
Trigram: ('dark', 'matter', 'show'), Count: 1218
Trigram: ('field', 'theories', 'theory'), Count: 1171
Trigram: ('consider', 'dark', 'matter'), Count: 1209
Trigram: ('dark', 'matter', 'range'), Count: 1008
Trigram: ('dark', 'matter', 'new'), Count: 1044
Trigram: ('Higgs', 'mass', 'model'), Count: 1033
Trigram: ('Higgs', 'boson', 'standard'), Count: 1481
Trigram: ('field', 'gauge', 'theory'), 

### **First Pass: Identifying Frequent Words**
1. **Tokenization and Counting:**  
   - Each abstract is split into individual words.  
   - The occurrences of each word across all abstracts are counted using PySpark transformations (`flatMap`, `map`, `reduceByKey`).  

2. **Filtering Frequent Words:**  
   - Words that appear fewer times than the specified `first_pass_support_threshold` (e.g., 300 occurrences) are discarded.  
   - A set of frequent words is returned, which is used in the second pass to restrict the scope of analysis.

### **Second Pass: Identifying Frequent Trigrams**
1. **Trigram Generation:**  
   - For each abstract, all possible trigrams (combinations of three unique words) are generated using Python's `itertools.combinations`.  
   - Only trigrams where all three words are in the frequent word set (from the first pass) are retained, ensuring efficiency.  

2. **Counting Trigram Occurrences:**  
   - The occurrences of each trigram across all abstracts are counted using similar transformations.  
   
3. **Filtering Frequent Trigrams:**  
   - Trigrams with occurrences below the `second_pass_support_threshold` (e.g., 1000) are discarded.  
   - The remaining frequent trigrams are collected as the final result.


## B:

In [10]:
# Parameters
support_threshold_first_pass = 300  # Lower support threshold for the first pass
support_threshold_second_pass = 1000  # Higher support threshold for the second pass
hash_table_size = 10**10  # Large hash table to reduce collisions

# Hash function to reduce collisions
def hash_trigram(trigram, hash_table_size):
    hash_obj = hashlib.sha256(str(trigram).encode())
    return int(hash_obj.hexdigest(), 16) % hash_table_size

# Normalize the trigram (sort to eliminate rotations)
def normalize_trigram(trigram):
    return tuple(sorted(trigram))

# First pass: Count frequent words and populate hash table
def first_pass(rdd, support_threshold_first_pass):
    # Count occurrences of words
    word_counts = rdd.flatMap(lambda record: record['abstract'].split()) \
                     .map(lambda word: (word, 1)) \
                     .reduceByKey(lambda x, y: x + y)
    
    # Find frequent words
    frequent_words = word_counts.filter(lambda x: x[1] >= support_threshold_first_pass) \
                                .map(lambda x: x[0]) \
                                .collect()
    frequent_word_set = set(frequent_words)

    # Generate trigrams, hash them, and populate the hash table
    hash_table = rdd.flatMap(
        lambda record: [
            hash_trigram(normalize_trigram(trigram), hash_table_size)
            for trigram in combinations(record['abstract'].split(), 3)
            if len(set(trigram)) == 3 and set(trigram).issubset(frequent_word_set)
        ]
    ).map(lambda bucket: (bucket, 1)) \
     .reduceByKey(lambda x, y: x + y) \
     .collectAsMap()

    # Create bitmap based on hash table counts
    bitmap = {bucket: (count >= support_threshold_first_pass) for bucket, count in hash_table.items()}
    return frequent_word_set, bitmap

# Second pass: Count candidate trigrams using the bitmap
def second_pass(rdd, frequent_word_set, bitmap, support_threshold_second_pass):
    def is_frequent_bucket(trigram):
        bucket = hash_trigram(normalize_trigram(trigram), hash_table_size)
        return bitmap.get(bucket, False)

    # Generate and filter candidate trigrams
    candidate_trigrams = rdd.flatMap(
        lambda record: [
            normalize_trigram(trigram)
            for trigram in combinations(record['abstract'].split(), 3)
            if len(set(trigram)) == 3 and set(trigram).issubset(frequent_word_set) and is_frequent_bucket(trigram)
        ]
    ).map(lambda trigram: (trigram, 1)) \
     .reduceByKey(lambda x, y: x + y)

    # Apply the second pass threshold
    frequent_trigrams_temp = candidate_trigrams.filter(lambda x: x[1] >= support_threshold_second_pass)
    return frequent_trigrams_temp


# First pass: Lower threshold to capture more candidate trigrams
frequent_word_set, bitmap = first_pass(sampled_rdd, support_threshold_first_pass)

# Second pass: Apply a higher threshold to finalize frequent trigrams
frequent_trigrams_PCY = second_pass(sampled_rdd, frequent_word_set, bitmap, support_threshold_second_pass)

# Collect and display results
results = frequent_trigrams_PCY.collect()
for trigram, count in results:
    print(f"Trigram: {trigram}, Count: {count}")


Trigram: ('dark', 'interactions', 'matter'), Count: 1682
Trigram: ('TeV', 'mass', 'search'), Count: 1084
Trigram: ('dark', 'matter', 'parameter'), Count: 1194
Trigram: ('GeV', 'boson', 'mass'), Count: 1220
Trigram: ('dark', 'matter', 'symmetry'), Count: 1381
Trigram: ('dark', 'matter', 'searches'), Count: 1510
Trigram: ('Higgs', 'boson', 'vector'), Count: 1132
Trigram: ('Higgs', 'TeV', 'boson'), Count: 1781
Trigram: ('LHC', 'dark', 'matter'), Count: 1109
Trigram: ('TeV', 'mass', 'model'), Count: 1049
Trigram: ('Higgs', 'boson', 'bosons'), Count: 1251
Trigram: ('dark', 'matter', 'two'), Count: 1616
Trigram: ('dark', 'matter', 'show'), Count: 1218
Trigram: ('field', 'theories', 'theory'), Count: 1171
Trigram: ('consider', 'dark', 'matter'), Count: 1209
Trigram: ('dark', 'matter', 'range'), Count: 1008
Trigram: ('dark', 'matter', 'new'), Count: 1044
Trigram: ('Higgs', 'mass', 'model'), Count: 1033
Trigram: ('Higgs', 'boson', 'standard'), Count: 1481
Trigram: ('field', 'gauge', 'theory'), 

### **Steps in the Algorithm**

#### **1. Parameters**
- **`support_threshold_first_pass`**: A lower threshold used in the first pass to identify potential candidates.
- **`support_threshold_second_pass`**: A higher threshold used in the second pass to filter the final frequent trigrams.
- **`hash_table_size`**: Size of the hash table to reduce collisions when hashing trigrams.

#### **2. Utility Functions**
- **`hash_trigram`**: Hashes a trigram using SHA-256 and maps it to a bucket in the hash table.
- **`normalize_trigram`**: Sorts the words in a trigram to ensure consistency, treating `(A, B, C)` and `(C, B, A)` as the same.


### **First Pass: Generating Frequent Words and Populating the Hash Table**
1. **Word Counting**:
   - Tokenizes abstracts into words and counts their occurrences using PySpark transformations.
   - Filters words with counts below `support_threshold_first_pass` and stores the frequent words in a set.

2. **Trigram Generation and Hashing**:
   - Generates all trigrams from abstracts.
   - Filters trigrams to include only those composed of frequent words from the first pass.
   - Hashes each trigram into a bucket using `hash_trigram` and populates the hash table with bucket counts.

3. **Bitmap Creation**:
   - Converts the hash table into a bitmap where each bucket is marked as frequent (`True`) if its count meets the `support_threshold_first_pass`. This bitmap reduces memory usage and helps prune infrequent trigrams early.


### **Second Pass: Filtering and Counting Candidate Trigrams**
1. **Candidate Filtering Using Bitmap**:
   - Generates trigrams from abstracts, ensuring:
     - All words are frequent (from the first pass).
     - The trigram maps to a bucket marked as frequent in the bitmap.

2. **Trigram Counting**:
   - Counts occurrences of the filtered trigrams across all abstracts.

3. **Final Filtering**:
   - Retains trigrams that meet the `support_threshold_second_pass`.

## C:

In [11]:
# Step 1: Convert the results of both algorithms into sets of trigrams
def extract_trigram_set(rdd):
    return set(rdd.map(lambda x: x[0]).collect())  # Collect only the trigrams

# Extract sets from both algorithms
apriori_trigram_set = extract_trigram_set(frequent_trigrams_apriori)  # Replace with your A-priori RDD
pcy_trigram_set = extract_trigram_set(frequent_trigrams_PCY)  # Replace with your PCY results

# Step 2: Compute the Jaccard similarity
intersection = apriori_trigram_set.intersection(pcy_trigram_set)
union = apriori_trigram_set.union(pcy_trigram_set)
jaccard_similarity = len(intersection) / len(union) if len(union) > 0 else 0

# Step 3: Display the result
print(f"Jaccard Similarity: {jaccard_similarity:.4f}")

Jaccard Similarity: 1.0000


# Part 2:

## Preprocessing:

In [5]:
arxiv_rdd = sc.textFile("MDA2024-Arxiv-Dataset.json")
arxiv_rdd.take(10)

['{"id":"0704.0001","submitter":"Pavel Nadolsky","authors":"C. Bal\\\\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","title":"Calculation of prompt diphoton production cross sections at Tevatron and\\n  LHC energies","comments":"37 pages, 15 figures; published version","journal-ref":"Phys.Rev.D76:013009,2007","doi":"10.1103/PhysRevD.76.013009","report-no":"ANL-HEP-PR-07-12","categories":"hep-ph","license":null,"abstract":"  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatro

In [6]:
# Parse the JSON string
def parse_json(line):
    return json.loads(line)

parsed_rdd = arxiv_rdd.map(parse_json).filter(lambda x: x is not None)

In [7]:
# Identify and remove or impute null values
def remove_nulls(record):
    # Check for null or empty fields in the record
    if all(record.values()):  # Ensure no field is None or empty
        return record
    else:
        return None  # Ignore records with null or empty fields

cleaned_rdd = parsed_rdd.filter(lambda x: remove_nulls(x) is not None)

In [8]:
# Define a set of stopwords for removal
stopwords = set(StopWordsRemover.loadDefaultStopWords("english"))

def remove_stopwords(record):
    for key, value in record.items():
        if isinstance(value, str):  # Process only text fields
            words = value.split()
            filtered_words = [word for word in words if word.lower() not in stopwords]
            record[key] = " ".join(filtered_words)
    return record

stopwords_removed_rdd = cleaned_rdd.map(remove_stopwords)

In [9]:
def remove_useless_characters(record):
    for key, value in record.items():
        if isinstance(value, str):  # Process only text fields
            # Remove special characters, retaining only alphanumeric and spaces
            record[key] = re.sub(r"[^a-zA-Z0-9\s]", "", value)
    return record

final_cleaned_rdd = stopwords_removed_rdd.map(remove_useless_characters)

# Save or view the final cleaned RDD
final_cleaned_rdd.take(1)  # View the first cleaned records

[{'id': '07040008',
  'submitter': 'Damian Swift',
  'authors': 'Damian C Swift',
  'title': 'Numerical solution shock ramp compression general material properties',
  'comments': 'Minor corrections',
  'journal-ref': 'Journal Applied Physics vol 104 073536 2008',
  'doi': '10106312975338',
  'report-no': 'LAUR072051 LLNLJRNL410358',
  'categories': 'condmatmtrlsci',
  'license': 'httparxivorglicensesnonexclusivedistrib10',
  'abstract': 'general formulation developed represent material models applications dynamic loading Numerical methods devised calculate response shock ramp compression ramp decompression generalizing previous solutions scalar equations state numerical methods found flexible robust matched analytic results high accuracy basic ramp shock solution methods coupled solve composite deformation paths shockinduced impacts shock interactions planar interface different materials calculations capture much physics typical material dynamics experiments without requiring spatiall

In [18]:
# Get a 1% sample from final_cleaned_rdd
sample_fraction = 0.5
sampled_rdd = final_cleaned_rdd.sample(withReplacement=False, fraction=sample_fraction, seed=42)

# Check how many records are in the sampled RDD
sampled_count = sampled_rdd.count()
print(f"Number of records in sampled RDD ({sample_fraction*100}%): {sampled_count}")

Number of records in sampled RDD (50.0%): 20394


## Code:

In [24]:
# Parameters
num_hashes = 1000  # Increased number of hash functions for MinHash
r = 10            # Rows per band
b = 100           # Number of bands (num_hashes = r * b)

# Step 1: Preprocessing
def tokenize(text):
    return text.lower().split() if text else []

def generate_shingles(tokens, k=5):
    if len(tokens) < k:
        return set()
    return set([''.join(tokens[i:i + k]) for i in range(len(tokens) - k + 1)])

# Step 2: Generate MinHash Signatures
def minhash_signature(shingles, hash_functions):
    if not shingles:
        return [float('inf')] * len(hash_functions)
    signature = []
    for a, b, p in hash_functions:
        min_hash = min(((a * hash(s) + b) % p) for s in shingles)
        signature.append(min_hash)
    return signature

# Generate hash functions
def generate_hash_functions(num_hashes):
    p = 2**31 - 1  # A large prime number
    hash_functions = [(random.randint(1, p), random.randint(0, p), p) for _ in range(num_hashes)]
    return hash_functions

# Step 3: Locality Sensitive Hashing
def lsh_buckets(signature, r, b):
    bands = []
    for i in range(b):
        start = i * r
        end = start + r
        band = tuple(signature[start:end])
        bands.append(hash(band))
    return bands

# Step 4: Retrieve Similar Articles
def jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2) if set1 and set2 else 0

def find_similar_articles(lsh_index, article_id, shingles, dataset, threshold=0.5):
    if article_id not in lsh_index:
        return []
    
    candidates = set()
    for band_hash in lsh_index[article_id]:
        if band_hash in reverse_index:
            candidates.update(reverse_index[band_hash])
    candidates.discard(article_id)
    
    similar_articles = []
    query_shingles = shingles.get(article_id, set())
    for candidate in candidates:
        sim = jaccard_similarity(query_shingles, shingles.get(candidate, set()))
        if sim >= threshold:
            similar_articles.append((candidate, sim))
    return sorted(similar_articles, key=lambda x: -x[1])

# Main Workflow
articles = sampled_rdd.map(lambda x: (x['id'], generate_shingles(tokenize(x.get('abstract', ''))))).filter(lambda x: x[1]).collect()
shingles = {article[0]: article[1] for article in articles}

# Generate MinHash signatures
hash_functions = generate_hash_functions(num_hashes)
signatures = {article_id: minhash_signature(shingle_set, hash_functions) for article_id, shingle_set in shingles.items()}

# Create LSH index
lsh_index = {}
reverse_index = {}
for article_id, signature in signatures.items():
    bands = lsh_buckets(signature, r, b)
    lsh_index[article_id] = bands
    for band in bands:
        if band not in reverse_index:
            reverse_index[band] = set()
        reverse_index[band].add(article_id)

# Find and print similarities for the first 6 papers
counter = 0
for article_id in shingles.keys():
    if counter >= 6:
        break
    similar_articles = find_similar_articles(lsh_index, article_id, shingles, articles)
    if similar_articles:
        print(f"\nQuery Article ID: {article_id}")
        print("Similar Articles:")
        for similar_id, similarity in similar_articles:
            print(f"Article ID: {similar_id}, Similarity: {similarity:.2f}")
        counter += 1



Query Article ID: 08033126
Similar Articles:
Article ID: 08033127, Similarity: 1.00

Query Article ID: 08033127
Similar Articles:
Article ID: 08033126, Similarity: 1.00

Query Article ID: 08040080
Similar Articles:
Article ID: 08040090, Similarity: 1.00
Article ID: 08040093, Similarity: 1.00

Query Article ID: 08040090
Similar Articles:
Article ID: 08040093, Similarity: 1.00
Article ID: 08040080, Similarity: 1.00

Query Article ID: 08040093
Similar Articles:
Article ID: 08040090, Similarity: 1.00
Article ID: 08040080, Similarity: 1.00

Query Article ID: 08042962
Similar Articles:
Article ID: 08042970, Similarity: 1.00
Article ID: 08042973, Similarity: 0.80


### **1. Preprocessing**
- **Tokenization**:
  - Splits the text of each paper (e.g., abstract) into tokens (words).
  - Converts all text to lowercase for case-insensitive comparisons.
  
- **Shingles Generation**:
  - Creates overlapping substrings (shingles) of length (k) from the tokens.
  - Example: For tokens ["`this`", "`is`", "`a`", "`test`"] and (k=3), shingles are: {"`thisisatest`"}


### **2. MinHashing**
  - **Generate Random Hash Functions**:
     - h(x) = (a.x + b) % p
  - **Create MinHash Signatures**:
     - For each paper's shingles, compute the minimum hash value under each of the hash functions.
     - A signature is an array of these minimum hash values.

MinHashing ensures that the similarity between two sets of shingles is preserved.

### **3. Locality-Sensitive Hashing (LSH)**
- **Purpose**:
  - Efficiently group and retrieve similar items by hashing similar MinHash signatures into the same buckets.

- **Steps**:
  1. **Split MinHash Signatures into Bands**:
     - Divide the signature into (b) bands, each containing (r) rows.
  2. **Hash Each Band**:
     - For each band (subarray of the signature), compute a hash.
     - Papers with the same hash for a band are likely to be similar.


### **4. Query for Similar Papers**
- **Jaccard Similarity**:
  - Measures the similarity between two sets of shingles: J(A, B) = Intersection(A,B) / Union(A,B)
  - If the similarity exceeds a threshold (e.g., 0.5 ), the papers are considered similar.

- **Steps**:
  1. Identify "candidate" papers using LSH:
     - For the query paper, look up its bands in the LSH index to find potential matches.
  2. Compute Jaccard Similarity for these candidates.


### **5. Main Workflow**
1. **Data Preparation**:
   - Tokenize and shingle all papers in the dataset.
   - Compute MinHash signatures and build the LSH index.
2. **Query Similarity**:
   - For each paper, retrieve candidate matches using LSH.
   - Calculate and rank similar papers based on Jaccard Similarity.
